In [1]:
import sys
sys.path.append('/SEE-MTDA/see')
from sc.mesher.mesher import Mesher
import numpy as np
import os
import glob
from pathlib import Path
from sc.datasets.shared_utils import *
import open3d as o3d
from easydict import EasyDict
import yaml
from PIL import Image

def cfg_from_yaml_file(cfg_file):
    cfg = EasyDict()
    print("\n----- Cfgs -----")
    with open(cfg_file, 'r') as f:
        yaml_config = yaml.safe_load(f)
        print(yaml.dump(yaml_config))

    cfg.update(EasyDict(yaml_config))

    return cfg

cfg_file = '/SEE-MTDA/see/sc/cfgs/KIT-DM-ORH005.yaml'
cfg = cfg_from_yaml_file(cfg_file)
mesher = Mesher(cfg=cfg, cfg_path=cfg_file)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.

----- Cfgs -----
DATASET:
  CAMERA_CHANNELS:
  - image_2
  CLASSES:
  - Car
  DET2D_MASK: true
  DET2D_MODEL: htc
  NAME: kitti
  POINT_CLOUD_RANGE:
  - -70
  - -70
  - -2
  - 70
  - 70
  - 4
  SHRINK_MASK_PERCENTAGE: 2
DATA_PATH: /SEE-MTDA/data/kitti
MESHER:
  CLUSTERING:
    EPS_SCALING: 5
    NAME: db_scan
  MESH_ALGORITHM:
    LOWER_RADIUS: 0.01
    NAME: ball_pivoting
    UPPER_RADIUS: 1.155
  MIN_LIDAR_PTS_TO_MESH: 30
  NAME: det_mesh
  REPLACE_OBJECT_WITH_MESH:
    POINT_DISTANCE_THRESH: 0.1
  SAMPLING:
    NAME: vres_ring_based_sampling
    OPTIMAL_RING_HEIGHT: 0.05
    TYPE: poisson
  VRES: 0.4

Loading masks...
loading annotations into memory...
Done (t=3.11s)
creating index...
index created!
KittiObjects initialised!

Mesher Initialised!


# Export points in GT box

This is to assess shape completion with perfect data

In [2]:
mesher.min_lidar_pts_to_mesh = 5

for sample_idx in range(mesher.data_obj.__len__()):
    pcd_gtboxes = mesher.get_pcd_gtboxes(sample_idx)

    for object_id in range(len(pcd_gtboxes['gt_boxes'])):    

        object_pcd = pcd_gtboxes['pcd'].crop(pcd_gtboxes['gt_boxes'][object_id])

        if len(object_pcd.points) < mesher.min_lidar_pts_to_mesh:
            continue

        box_pts = np.asarray(pcd_gtboxes['gt_boxes'][object_id].get_box_points())

        if mesher.data_obj.dataset_name == 'nuscenes':
            fname = f'nsweeps-{mesher.data_obj.nsweeps}_frame-{sample_idx}_object-{object_id}_car_pts-{len(object_pcd.points)}'
        else:
            fname = f'frame-{sample_idx}_object-{object_id}_car_pts-{len(object_pcd.points)}'

        car_dir = Path(f'{mesher.data_obj.root_dir}/exported/cars')
        car_dir.mkdir(parents=True, exist_ok=True)
        o3d.io.write_point_cloud(str(car_dir / f'{fname}.pcd'), object_pcd)

        box_dir = Path(f'{mesher.data_obj.root_dir}/exported/bboxes')    
        box_dir.mkdir(parents=True, exist_ok=True)
        np.savetxt(str(box_dir / f'{fname}.txt'), box_pts, delimiter=' ')

        print(f"frame-{sample_idx}: Exported {object_id}/{len(pcd_gtboxes['gt_boxes'])} | car_pts-{len(object_pcd.points)}")


frame-1: Exported 0/1 | car_pts-9
frame-2: Exported 0/1 | car_pts-53


KeyboardInterrupt: 

# Export points in mask/bbox

This is to test out lidar segmentation on the points within a box

In [ ]:
from PIL import Image

camera_channel = 'image_2'

export_tag = 'bbox'
if export_tag == 'bbox':
    use_bbox = True
else:
    use_bbox = False

instance_dataset_id = 0
for sample_idx in range(mesher.data_obj.__len__()):
    
    i_clouds = mesher.data_obj.get_mask_instance_clouds(sample_idx, use_bbox=use_bbox)
    pcd_gtboxes = mesher.get_pcd_gtboxes(sample_idx, add_ground_lift=True)    
    inst_labels = mesher.data_obj.get_camera_instances(sample_idx, channel=camera_channel)
        
    for object_id, i_cloud in enumerate(i_clouds):
        mask_pcd = convert_to_o3dpcd(i_clouds[object_id])
        cropped_pcds, num_pts_in_gt_for_mask = [], []        
        
        if len(i_cloud) < 5:
            continue
            
        for i in range(len(pcd_gtboxes['gt_boxes'])):    
            cropped = mask_pcd.crop(pcd_gtboxes['gt_boxes'][i])
            num_pts_in_gt_for_mask.append(len(cropped.points))
            cropped_pcds.append(cropped)            
            
        if not num_pts_in_gt_for_mask or max(num_pts_in_gt_for_mask) == 0:
            labelled_pts = np.hstack([i_cloud[:,:3], np.zeros((len(i_cloud),1))])
            box_pts = np.zeros((8,3))
            num_points_in_gt = 0
        else:
            gtbox_idx = np.argmax(np.array(num_pts_in_gt_for_mask)) 
            
            cropped_np = np.array(cropped_pcds[gtbox_idx].points)
            obj_pts_labelled = np.hstack([cropped_np, 10*np.ones((len(cropped_np),1))])

            dist = mask_pcd.compute_point_cloud_distance(cropped_pcds[gtbox_idx])
            cropped_inds = np.where(np.asarray(dist) < 0.01)[0]
            mask_without_obj = np.asarray(mask_pcd.select_by_index(cropped_inds, invert=True).points)
            mask_pts_labelled = np.hstack([mask_without_obj, np.zeros((len(mask_without_obj),1))])

            labelled_pts = np.vstack([mask_pts_labelled, obj_pts_labelled])
            box_pts = np.asarray(pcd_gtboxes['gt_boxes'][gtbox_idx].get_box_points())
            num_points_in_gt = len(cropped_np)
               
        # Save pcl
        fname = f"{instance_dataset_id:06}"
        car_dir = Path(f'{mesher.data_obj.root_dir}/exported/lidar_seg_data_groundlift/points_in_{export_tag}/pointcloud')
        car_dir.mkdir(parents=True, exist_ok=True)
        car_fname = str(car_dir / f'{fname}.bin')
        points = np.float32(labelled_pts)
        points.tofile(car_fname)
        
        # Save points labels
        label_dir = Path(f'{mesher.data_obj.root_dir}/exported/lidar_seg_data_groundlift/points_in_{export_tag}/labels')
        label_dir.mkdir(parents=True, exist_ok=True)
        label_fname = str(label_dir / f'{fname}.label')
        labels = np.uint32(labelled_pts[:,3])
        labels.tofile(label_fname)
        
        # Save gt bboxes
        box_dir = Path(f'{mesher.data_obj.root_dir}/exported/lidar_seg_data_groundlift/points_in_{export_tag}/gt_bboxes')    
        box_dir.mkdir(parents=True, exist_ok=True)
        np.savetxt(str(box_dir / f'{fname}.txt'), box_pts, delimiter=' ')
        
        # Save img
        img_dir = Path(f'{mesher.data_obj.root_dir}/exported/lidar_seg_data_groundlift/points_in_{export_tag}/image')    
        img_dir.mkdir(parents=True, exist_ok=True)
        img = mesher.data_obj.get_image(sample_idx, channel=camera_channel)
        imglabels = inst_labels[object_id]
        ann_img = draw_lidar_on_image([], img, instances=[imglabels], display=False)
        Image.fromarray(ann_img).save(str(img_dir / f'{fname}.png'))

        # Save original filepath of input pointcloud
        origpath_dir = Path(f'{mesher.data_obj.root_dir}/exported/lidar_seg_data_groundlift/points_in_{export_tag}/fpath')
        origpath_dir.mkdir(parents=True, exist_ok=True)
        with open(str(origpath_dir / f'{fname}.txt'), 'w') as f:
              f.write(f'{sample_idx:06}')
        
        print(f"car #{instance_dataset_id}: Exported {object_id+1}/{len(i_clouds)} | total pts-{len(labelled_pts)}, gt pts - {num_points_in_gt}")
        instance_dataset_id = instance_dataset_id + 1

car #0: Exported 1/1 | total pts-10, gt pts - 9
car #1: Exported 1/1 | total pts-110, gt pts - 53
car #2: Exported 1/4 | total pts-408, gt pts - 0
car #3: Exported 2/4 | total pts-952, gt pts - 618
car #4: Exported 4/4 | total pts-15, gt pts - 0
car #5: Exported 1/3 | total pts-120, gt pts - 65
car #6: Exported 2/3 | total pts-33, gt pts - 25
car #7: Exported 1/5 | total pts-666, gt pts - 282
car #8: Exported 2/5 | total pts-138, gt pts - 53
car #9: Exported 3/5 | total pts-98, gt pts - 15
car #10: Exported 4/5 | total pts-10, gt pts - 0
car #11: Exported 5/5 | total pts-16, gt pts - 0
car #12: Exported 1/3 | total pts-213, gt pts - 156
car #13: Exported 2/3 | total pts-25, gt pts - 18
car #14: Exported 3/3 | total pts-7, gt pts - 5
car #15: Exported 1/10 | total pts-2959, gt pts - 1445
car #16: Exported 2/10 | total pts-1579, gt pts - 843
car #17: Exported 3/10 | total pts-3099, gt pts - 1529
car #18: Exported 4/10 | total pts-1038, gt pts - 590
car #19: Exported 5/10 | total pts-298,

car #160: Exported 2/7 | total pts-345, gt pts - 160
car #161: Exported 3/7 | total pts-113, gt pts - 60
car #162: Exported 4/7 | total pts-102, gt pts - 26
car #163: Exported 5/7 | total pts-44, gt pts - 18
car #164: Exported 6/7 | total pts-25, gt pts - 20
car #165: Exported 7/7 | total pts-10, gt pts - 0
car #166: Exported 1/6 | total pts-3209, gt pts - 1829
car #167: Exported 2/6 | total pts-640, gt pts - 389
car #168: Exported 3/6 | total pts-128, gt pts - 102
car #169: Exported 4/6 | total pts-136, gt pts - 94
car #170: Exported 5/6 | total pts-55, gt pts - 21
car #171: Exported 6/6 | total pts-49, gt pts - 22
car #172: Exported 1/9 | total pts-1543, gt pts - 931
car #173: Exported 2/9 | total pts-527, gt pts - 202
car #174: Exported 3/9 | total pts-190, gt pts - 119
car #175: Exported 4/9 | total pts-138, gt pts - 23
car #176: Exported 5/9 | total pts-169, gt pts - 134
car #177: Exported 6/9 | total pts-153, gt pts - 88
car #178: Exported 7/9 | total pts-25, gt pts - 12
car #179

car #318: Exported 1/8 | total pts-1433, gt pts - 223
car #319: Exported 2/8 | total pts-1854, gt pts - 60
car #320: Exported 3/8 | total pts-1318, gt pts - 399
car #321: Exported 4/8 | total pts-201, gt pts - 46
car #322: Exported 5/8 | total pts-216, gt pts - 53
car #323: Exported 6/8 | total pts-77, gt pts - 30
car #324: Exported 7/8 | total pts-48, gt pts - 8
car #325: Exported 8/8 | total pts-31, gt pts - 0
car #326: Exported 1/2 | total pts-156, gt pts - 120
car #327: Exported 2/2 | total pts-14, gt pts - 2
car #328: Exported 1/7 | total pts-1646, gt pts - 645
car #329: Exported 2/7 | total pts-156, gt pts - 48
car #330: Exported 3/7 | total pts-476, gt pts - 324
car #331: Exported 4/7 | total pts-451, gt pts - 176
car #332: Exported 5/7 | total pts-357, gt pts - 182
car #333: Exported 6/7 | total pts-184, gt pts - 76
car #334: Exported 7/7 | total pts-97, gt pts - 25
car #335: Exported 1/6 | total pts-844, gt pts - 424
car #336: Exported 2/6 | total pts-77, gt pts - 53
car #337:

car #476: Exported 5/11 | total pts-647, gt pts - 282
car #477: Exported 6/11 | total pts-400, gt pts - 189
car #478: Exported 7/11 | total pts-179, gt pts - 60
car #479: Exported 8/11 | total pts-328, gt pts - 134
car #480: Exported 9/11 | total pts-179, gt pts - 88
car #481: Exported 10/11 | total pts-87, gt pts - 46
car #482: Exported 11/11 | total pts-57, gt pts - 11
car #483: Exported 1/8 | total pts-3045, gt pts - 1815
car #484: Exported 2/8 | total pts-472, gt pts - 286
car #485: Exported 3/8 | total pts-367, gt pts - 155
car #486: Exported 4/8 | total pts-219, gt pts - 131
car #487: Exported 5/8 | total pts-153, gt pts - 76
car #488: Exported 6/8 | total pts-176, gt pts - 62
car #489: Exported 7/8 | total pts-116, gt pts - 68
car #490: Exported 8/8 | total pts-68, gt pts - 29
car #491: Exported 1/11 | total pts-518, gt pts - 367
car #492: Exported 2/11 | total pts-190, gt pts - 0
car #493: Exported 3/11 | total pts-439, gt pts - 150
car #494: Exported 4/11 | total pts-188, gt p

car #634: Exported 3/4 | total pts-160, gt pts - 63
car #635: Exported 4/4 | total pts-105, gt pts - 25
car #636: Exported 1/1 | total pts-888, gt pts - 504
car #637: Exported 1/10 | total pts-2860, gt pts - 1878
car #638: Exported 2/10 | total pts-1431, gt pts - 489
car #639: Exported 3/10 | total pts-1277, gt pts - 685
car #640: Exported 4/10 | total pts-774, gt pts - 346
car #641: Exported 5/10 | total pts-580, gt pts - 146
car #642: Exported 6/10 | total pts-503, gt pts - 299
car #643: Exported 7/10 | total pts-491, gt pts - 247
car #644: Exported 8/10 | total pts-212, gt pts - 126
car #645: Exported 9/10 | total pts-51, gt pts - 28
car #646: Exported 10/10 | total pts-83, gt pts - 40
car #647: Exported 1/6 | total pts-130, gt pts - 11
car #648: Exported 2/6 | total pts-41, gt pts - 13
car #649: Exported 3/6 | total pts-40, gt pts - 28
car #650: Exported 1/3 | total pts-1348, gt pts - 262
car #651: Exported 2/3 | total pts-2139, gt pts - 1339
car #652: Exported 3/3 | total pts-191,

...

car #9452: Exported 3/4 | total pts-71, gt pts - 41
car #9453: Exported 1/3 | total pts-1386, gt pts - 350
car #9454: Exported 2/3 | total pts-829, gt pts - 453
car #9455: Exported 3/3 | total pts-129, gt pts - 61
car #9456: Exported 1/5 | total pts-1360, gt pts - 746
car #9457: Exported 2/5 | total pts-163, gt pts - 0
car #9458: Exported 3/5 | total pts-19, gt pts - 5
car #9459: Exported 4/5 | total pts-38, gt pts - 28
car #9460: Exported 5/5 | total pts-8, gt pts - 1
car #9461: Exported 1/1 | total pts-1876, gt pts - 0
car #9462: Exported 1/3 | total pts-654, gt pts - 180
car #9463: Exported 2/3 | total pts-37, gt pts - 24
car #9464: Exported 1/16 | total pts-248, gt pts - 136
car #9465: Exported 2/16 | total pts-540, gt pts - 176
car #9466: Exported 3/16 | total pts-70, gt pts - 14
car #9467: Exported 4/16 | total pts-192, gt pts - 72
car #9468: Exported 5/16 | total pts-262, gt pts - 161
car #9469: Exported 6/16 | total pts-90, gt pts - 27
car #9470: Exported 7/16 | total pts-123, 

car #9606: Exported 5/13 | total pts-226, gt pts - 126
car #9607: Exported 6/13 | total pts-142, gt pts - 46
car #9608: Exported 7/13 | total pts-101, gt pts - 46
car #9609: Exported 8/13 | total pts-34, gt pts - 31
car #9610: Exported 9/13 | total pts-23, gt pts - 20
car #9611: Exported 10/13 | total pts-9, gt pts - 3
car #9612: Exported 11/13 | total pts-21, gt pts - 7
car #9613: Exported 12/13 | total pts-17, gt pts - 14
car #9614: Exported 13/13 | total pts-15, gt pts - 12
car #9615: Exported 1/1 | total pts-1507, gt pts - 0
car #9616: Exported 1/4 | total pts-356, gt pts - 108
car #9617: Exported 2/4 | total pts-139, gt pts - 84
car #9618: Exported 3/4 | total pts-50, gt pts - 39
car #9619: Exported 4/4 | total pts-5, gt pts - 0
car #9620: Exported 1/3 | total pts-2578, gt pts - 478
car #9621: Exported 2/3 | total pts-59, gt pts - 27
car #9622: Exported 3/3 | total pts-42, gt pts - 29
car #9623: Exported 1/2 | total pts-1222, gt pts - 247
car #9624: Exported 2/2 | total pts-432, g

car #9760: Exported 4/7 | total pts-823, gt pts - 313
car #9761: Exported 5/7 | total pts-492, gt pts - 312
car #9762: Exported 6/7 | total pts-272, gt pts - 111
car #9763: Exported 7/7 | total pts-19, gt pts - 0
car #9764: Exported 1/2 | total pts-1571, gt pts - 745
car #9765: Exported 2/2 | total pts-103, gt pts - 35
car #9766: Exported 1/14 | total pts-4391, gt pts - 81
car #9767: Exported 2/14 | total pts-580, gt pts - 311
car #9768: Exported 3/14 | total pts-948, gt pts - 572
car #9769: Exported 4/14 | total pts-410, gt pts - 165
car #9770: Exported 5/14 | total pts-241, gt pts - 32
car #9771: Exported 6/14 | total pts-121, gt pts - 30
car #9772: Exported 7/14 | total pts-314, gt pts - 189
car #9773: Exported 8/14 | total pts-258, gt pts - 159
car #9774: Exported 9/14 | total pts-175, gt pts - 96
car #9775: Exported 10/14 | total pts-45, gt pts - 36
car #9776: Exported 11/14 | total pts-107, gt pts - 26
car #9777: Exported 12/14 | total pts-61, gt pts - 31
car #9778: Exported 13/1

car #9914: Exported 7/8 | total pts-46, gt pts - 4
car #9915: Exported 8/8 | total pts-60, gt pts - 10
car #9916: Exported 1/16 | total pts-2151, gt pts - 870
car #9917: Exported 2/16 | total pts-1366, gt pts - 898
car #9918: Exported 3/16 | total pts-719, gt pts - 390
car #9919: Exported 4/16 | total pts-657, gt pts - 389
car #9920: Exported 5/16 | total pts-769, gt pts - 222
car #9921: Exported 6/16 | total pts-382, gt pts - 223
car #9922: Exported 7/16 | total pts-459, gt pts - 192
car #9923: Exported 8/16 | total pts-146, gt pts - 45
car #9924: Exported 9/16 | total pts-143, gt pts - 55
car #9925: Exported 10/16 | total pts-80, gt pts - 53
car #9926: Exported 11/16 | total pts-54, gt pts - 22
car #9927: Exported 12/16 | total pts-59, gt pts - 21
car #9928: Exported 14/16 | total pts-5, gt pts - 0
car #9929: Exported 1/8 | total pts-3400, gt pts - 2064
car #9930: Exported 2/8 | total pts-1682, gt pts - 1135
car #9931: Exported 3/8 | total pts-1424, gt pts - 906
car #9932: Exported 4

car #10067: Exported 6/15 | total pts-453, gt pts - 195
car #10068: Exported 7/15 | total pts-398, gt pts - 184
car #10069: Exported 8/15 | total pts-159, gt pts - 82
car #10070: Exported 9/15 | total pts-251, gt pts - 94
car #10071: Exported 10/15 | total pts-107, gt pts - 47
car #10072: Exported 11/15 | total pts-87, gt pts - 39
car #10073: Exported 12/15 | total pts-14, gt pts - 0
car #10074: Exported 13/15 | total pts-7, gt pts - 0
car #10075: Exported 14/15 | total pts-5, gt pts - 0
car #10076: Exported 1/13 | total pts-1751, gt pts - 923
car #10077: Exported 2/13 | total pts-861, gt pts - 394
car #10078: Exported 3/13 | total pts-1482, gt pts - 534
car #10079: Exported 4/13 | total pts-717, gt pts - 404
car #10080: Exported 5/13 | total pts-848, gt pts - 416
car #10081: Exported 6/13 | total pts-1058, gt pts - 306
car #10082: Exported 7/13 | total pts-487, gt pts - 126
car #10083: Exported 8/13 | total pts-258, gt pts - 82
car #10084: Exported 9/13 | total pts-129, gt pts - 31
ca